In [2]:
import json
import os
import pandas as pd

In [3]:
os.chdir('../')

## Connect to the server

In [4]:
filename_credentials = 'clickhouse_credentials.json'

In [5]:
with open(filename_credentials, 'r') as f:
    config = json.load(f)

In [6]:
config

{'host': '35.210.233.219',
 'port': 9001,
 'user': 'user1',
 'password': 'hola1234',
 'database': 'default'}

In [7]:
from src.data.load_data import connect_clickhouse

In [8]:
client = connect_clickhouse(config)

In [9]:
client.execute('SHOW TABLES')

[array(['test'], dtype=object), array(['test_pandas'], dtype=object)]

## Create a schema

In [10]:
database = 'raw'

In [11]:
client.execute('CREATE DATABASE IF NOT EXISTS ' + database)

[]

In [12]:
client.execute('SHOW DATABASES')

[array(['_temporary_and_external_tables'], dtype=object),
 array(['default'], dtype=object),
 array(['raw'], dtype=object),
 array(['system'], dtype=object)]

## Insert dataframe

In [13]:
table_name = 'meal'
file_path = '../api/data/raw/meal_info.csv'

In [14]:
df_meal = pd.read_csv(file_path)

In [15]:
df_meal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   meal_id   51 non-null     int64 
 1   category  51 non-null     object
 2   cuisine   51 non-null     object
dtypes: int64(1), object(2)
memory usage: 1.3+ KB


In [17]:
client.execute('DROP TABLE IF EXISTS default.test_pandas')

[]

In [18]:
client.execute('''
    CREATE TABLE IF NOT EXISTS default.test_pandas
    (
        meal_id Int64,
        category String,
        cuisine String
    )
    ENGINE = Memory
''')

[]

In [19]:
client.insert_dataframe('INSERT INTO default.test_pandas VALUES', df_meal)

51

In [20]:
client.query_dataframe('SELECT * FROM default.test_pandas')

,meal_id,category,cuisine
0,1885,Beverages,Thai
1,1993,Beverages,Thai
2,2539,Beverages,Thai
3,1248,Beverages,Indian
4,2631,Beverages,Indian
5,1311,Extras,Thai
6,1062,Beverages,Italian
7,1778,Beverages,Italian
8,1803,Extras,Thai
9,1198,Extras,Thai
